In [1]:
import pandas as pd
import os
from keplergl import KeplerGl 
from pyproj import CRS
import numpy as np
from matplotlib import pyplot as plt

C:\Users\stefa\anaconda3\envs\CitiBike\Lib\site-packages\keplergl\keplergl.py:2: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import resource_string


In [2]:
path = r"C:\Users\stefa\CitiBike"

In [3]:
df = pd.read_csv(os.path.join(path, 'Prepared Data', 'CitiBikeWeatherWrangled.csv'),  index_col = False)

Step 3: In the data set from the previous task, create a new column with the value of 1. Then create a new aggregated dataframe that contains 3 columns: starting station, ending station, and the count of trips between those stations. 

In [4]:
df.columns

Index(['date', 'ride_id', 'rideable_type', 'started_at', 'ended_at',
       'start_station_name', 'start_station_id', 'end_station_name',
       'end_station_id', 'start_lat', 'start_lng', 'end_lat', 'end_lng',
       'member_casual', 'average_tempurature', 'bike_rides_daily', '_merge'],
      dtype='object')

In [5]:
df = df.dropna()

In [6]:
df['value'] = 1
df_group = df.groupby(['start_station_name', 'end_station_name'])['value'].count().reset_index()

In [7]:
df_group

,start_station_name,end_station_name,value
0,11 St & Washington St,11 St & Washington St,1132
1,11 St & Washington St,12 Ave & W 40 St,1
2,11 St & Washington St,12 St & Sinatra Dr N,253
3,11 St & Washington St,14 St Ferry - 14 St & Shipyard Ln,395
4,11 St & Washington St,4 St & Grand St,350
...,...,...,...
6948,York St & Marin Blvd,Van Vorst Park,18
6949,York St & Marin Blvd,Warren St,42
6950,York St & Marin Blvd,Washington St,16
6951,York St & Marin Blvd,Willow Ave & 12 St,1


In [8]:
print(df_group['value'].sum())
print(df.shape)

892281
(892281, 18)


In [9]:
df_group.rename(columns = {'value': 'trips'}, inplace = True)

In [10]:
df_group

,start_station_name,end_station_name,trips
0,11 St & Washington St,11 St & Washington St,1132
1,11 St & Washington St,12 Ave & W 40 St,1
2,11 St & Washington St,12 St & Sinatra Dr N,253
3,11 St & Washington St,14 St Ferry - 14 St & Shipyard Ln,395
4,11 St & Washington St,4 St & Grand St,350
...,...,...,...
6948,York St & Marin Blvd,Van Vorst Park,18
6949,York St & Marin Blvd,Warren St,42
6950,York St & Marin Blvd,Washington St,16
6951,York St & Marin Blvd,Willow Ave & 12 St,1


Step 4: Initialize an instance of a kepler.gl map. 

In [11]:
df_stations = df[['start_station_name', 'end_station_name', 'start_lat', 'start_lng', 'end_lat', 'end_lng']]

In [12]:
df_stations

,start_station_name,end_station_name,start_lat,start_lng,end_lat,end_lng
0,Newport PATH,Newport PATH,40.727224,-74.033759,40.727224,-74.033759
1,Bergen Ave,Washington St,40.722104,-74.071455,40.724294,-74.035483
2,Sip Ave,Bergen Ave,40.730897,-74.063913,40.722104,-74.071455
3,Newport Pkwy,Jersey & 3rd,40.728745,-74.032108,40.723332,-74.045953
4,Columbus Dr at Exchange Pl,Jersey & 3rd,40.716870,-74.032810,40.723332,-74.045953
...,...,...,...,...,...,...
895480,Bergen Ave & Stegman St,Pershing Field,40.706717,-74.086701,40.742677,-74.051789
895481,City Hall - Washington St & 1 St,11 St & Washington St,40.737360,-74.030970,40.749985,-74.027150
895482,9 St HBLR - Jackson St & 8 St,12 St & Sinatra Dr N,40.747907,-74.038412,40.750604,-74.024020
895483,Grove St PATH,Dixon Mills,40.719410,-74.043090,40.721630,-74.049968


df_m = df_group.merge(df_stations, how = 'outer', on = "start_station_name", indicator = 'merge_flag')

In [13]:
df_m = pd.merge(df_group, df_stations, on=['start_station_name', 'end_station_name'], how='outer', indicator = 'merge_flag')

In [14]:
df_m['merge_flag'].value_counts(dropna = True)

merge_flag
both          892281
left_only          0
right_only         0
Name: count, dtype: int64

In [15]:
df_m.shape

(892281, 8)

In [16]:
df_m.head()

,start_station_name,end_station_name,trips,start_lat,start_lng,end_lat,end_lng,merge_flag
0,11 St & Washington St,11 St & Washington St,1132,40.749985,-74.02715,40.749985,-74.02715,both
1,11 St & Washington St,11 St & Washington St,1132,40.749985,-74.02715,40.749985,-74.02715,both
2,11 St & Washington St,11 St & Washington St,1132,40.749985,-74.02715,40.749985,-74.02715,both
3,11 St & Washington St,11 St & Washington St,1132,40.749985,-74.02715,40.749985,-74.02715,both
4,11 St & Washington St,11 St & Washington St,1132,40.749985,-74.02715,40.749985,-74.02715,both


In [17]:
df_m.drop(columns = ['merge_flag'])

,start_station_name,end_station_name,trips,start_lat,start_lng,end_lat,end_lng
0,11 St & Washington St,11 St & Washington St,1132,40.749985,-74.027150,40.749985,-74.027150
1,11 St & Washington St,11 St & Washington St,1132,40.749985,-74.027150,40.749985,-74.027150
2,11 St & Washington St,11 St & Washington St,1132,40.749985,-74.027150,40.749985,-74.027150
3,11 St & Washington St,11 St & Washington St,1132,40.749985,-74.027150,40.749985,-74.027150
4,11 St & Washington St,11 St & Washington St,1132,40.749985,-74.027150,40.749985,-74.027150
...,...,...,...,...,...,...,...
892276,York St & Marin Blvd,York St & Marin Blvd,47,40.716615,-74.042412,40.716615,-74.042412
892277,York St & Marin Blvd,York St & Marin Blvd,47,40.716615,-74.042412,40.716615,-74.042412
892278,York St & Marin Blvd,York St & Marin Blvd,47,40.716615,-74.042412,40.716615,-74.042412
892279,York St & Marin Blvd,York St & Marin Blvd,47,40.716615,-74.042412,40.716615,-74.042412


In [18]:
import gc # this is garbage collector - speeds up performance
gc.collect()

209

In [19]:
indices_to_drop = df_m[df_m['trips'] < 100].index

In [20]:
df_m = df_m.drop(indices_to_drop)

In [21]:
df_m = df_m.reset_index(drop=True)

In [22]:
df_m

,start_station_name,end_station_name,trips,start_lat,start_lng,end_lat,end_lng,merge_flag
0,11 St & Washington St,11 St & Washington St,1132,40.749985,-74.027150,40.749985,-74.027150,both
1,11 St & Washington St,11 St & Washington St,1132,40.749985,-74.027150,40.749985,-74.027150,both
2,11 St & Washington St,11 St & Washington St,1132,40.749985,-74.027150,40.749985,-74.027150,both
3,11 St & Washington St,11 St & Washington St,1132,40.749985,-74.027150,40.749985,-74.027150,both
4,11 St & Washington St,11 St & Washington St,1132,40.749985,-74.027150,40.749985,-74.027150,both
...,...,...,...,...,...,...,...,...
772611,Willow Ave & 12 St,Willow Ave & 12 St,494,40.751867,-74.030377,40.751867,-74.030377,both
772612,Willow Ave & 12 St,Willow Ave & 12 St,494,40.751867,-74.030377,40.751867,-74.030377,both
772613,Willow Ave & 12 St,Willow Ave & 12 St,494,40.751975,-74.030357,40.751867,-74.030377,both
772614,Willow Ave & 12 St,Willow Ave & 12 St,494,40.751867,-74.030377,40.751867,-74.030377,both


In [23]:
df_m.to_csv(os.path.join(path, 'Prepared Data', 'CitiBike_final_locations_for_map_filtered.csv'))